In [7]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

import warnings
warnings.filterwarnings("ignore")

In [17]:
company_dict = {"미래에셋증권":"006800","키움증권" : "039490" , "NH투자증권": "005940","삼성증권" : "016360" }
df = pd.DataFrame(list(company_dict.items()),columns = ['company','code'])
df

,company,code
0,미래에셋증권,006800
1,키움증권,039490
2,NH투자증권,005940
3,삼성증권,016360


In [24]:
for com in range(len(df)):
    print(df.loc[com,'company'], df.loc[com,'code'])

미래에셋증권 006800
키움증권 039490
NH투자증권 005940
삼성증권 016360


In [26]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

import warnings
warnings.filterwarnings("ignore")

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    




company_dict = {"미래에셋증권":"006800","키움증권" : "039490" , "NH투자증권": "005940","삼성증권" : "016360" }
df_company = pd.DataFrame(list(company_dict.items()),columns = ['company','code'])
df_company


for com in range(len(df_company)):
    print(com)
    
    df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실'  and RELATED_ITEM = '{}'".format(df.loc[com,'company']),conn)
    df_already

    df_ttl = pd.DataFrame()
    board_nm = 0    
    while True:
        board_nm +=1
        item_board_url = "https://finance.naver.com/item/board.naver?code=" + df.loc[com,'code'] +"&page=" + str(board_nm)
        print(item_board_url)
        res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
        soup = BeautifulSoup(res.text, "html5lib")
        #         tab_title = soup.findAll('td', attrs={'class':'title'})   
        tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
        tab_date_txt = [i.text for i in tab_date]

        tab_title = soup.findAll('td', attrs={'class':'title'})  
        tab_title_txt = [i.text for i in tab_title]

        tab_writer = soup.findAll('td', attrs={'class':'p11'})  
        tab_writer_txt = [i.text for i in tab_writer]
        tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

        url_l = []
        for n in soup.findAll('td', attrs={'class':'title'})  :
            url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
        url_l

        like_l = []
        for s in soup.findAll('tr'): 
            if len(s.findAll('strong')) > 0:
                like_l.append(s.findAll('strong')[0].text)
        like_l[:20]


        df_ = pd.DataFrame()
        df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
        df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
        df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
        df_['LIKES'] = like_l[:20]
        df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
        df_['CONTENT'] = np.nan
        df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

        df_['SOURCE'] = "네이버_종목토론실"
        df_['RELATED_ITEM'] = df.loc[com,'company']
        df_['POSITIVITY'] = np.nan
        df_['OTHERS'] = df_[['WRITER']].to_dict('records')
        df_['OTHERS'] = df_['OTHERS'].astype(str)
        df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
        df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
        df_['URL'] =  url_l
        df_


        for i in range(len(df_)):
            df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])


        df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
        df_
        df_ttl = df_ttl.append(df_)
        if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
            break     
    #     print(board_nm)

    df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
    df_ttl   

    if len(df_ttl) >0:
        df_ttl = df_ttl.reset_index(drop=True)

        df_ttl
        df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
        print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , "네이버종토_{} SUCCESS  : ".format(df.loc[com,'company']),len(df_ttl))
    else:
        print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), "네이버종토_{} NO DATA TO UPDATE".format(df.loc[com,'company']))





0
https://finance.naver.com/item/board.naver?code=006800&page=1
20220922-094623 네이버종토_미래에셋증권 SUCCESS  :  1
1
https://finance.naver.com/item/board.naver?code=039490&page=1
https://finance.naver.com/item/board.naver?code=039490&page=2
https://finance.naver.com/item/board.naver?code=039490&page=3
https://finance.naver.com/item/board.naver?code=039490&page=4
20220922-094631 네이버종토_키움증권 SUCCESS  :  60
2
https://finance.naver.com/item/board.naver?code=005940&page=1
https://finance.naver.com/item/board.naver?code=005940&page=2
https://finance.naver.com/item/board.naver?code=005940&page=3
20220922-094637 네이버종토_NH투자증권 SUCCESS  :  40
3
https://finance.naver.com/item/board.naver?code=016360&page=1
https://finance.naver.com/item/board.naver?code=016360&page=2
https://finance.naver.com/item/board.naver?code=016360&page=3
https://finance.naver.com/item/board.naver?code=016360&page=4
https://finance.naver.com/item/board.naver?code=016360&page=5
https://finance.naver.com/item/board.naver?code=016360&pa

In [26]:
(datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')

'20220830-082328'

In [11]:
def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' ORDER BY DATETIME desc",conn)
df_already


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220825-141100,오늘 폭등 무슨 호재 있나여?[2],None,0.281904,2,https://finance.naver.com//item/board_read.nav...,20220825-052830,{'WRITER': 'asse****'}
1,네이버_종목토론실,미래에셋증권,20220825-141100,오늘 폭등 무슨 호재 있나여?[2],None,0.281904,2,https://finance.naver.com//item/board_read.nav...,20220825-052903,{'WRITER': 'asse****'}
2,네이버_종목토론실,미래에셋증권,20220825-122100,이 엿같은 회사야 지수도 무시하냐?,None,0.005595,0,https://finance.naver.com//item/board_read.nav...,20220825-052830,{'WRITER': 'pdy1****'}
3,네이버_종목토론실,미래에셋증권,20220825-122100,이 엿같은 회사야 지수도 무시하냐?,None,0.005595,0,https://finance.naver.com//item/board_read.nav...,20220825-052903,{'WRITER': 'pdy1****'}
4,네이버_종목토론실,미래에셋증권,20220825-122000,2만 원 적정가야~~~~시총 8조이상 가...,None,0.244802,0,https://finance.naver.com//item/board_read.nav...,20220825-052830,{'WRITER': 'hjt4****'}
...,...,...,...,...,...,...,...,...,...,...
2141,네이버_종목토론실,미래에셋증권,20220101-090900,올해는 개행주 현만이의 역대급저주로,None,0.009513,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}
2142,네이버_종목토론실,미래에셋증권,20211231-102600,배당금 얼마 주나요?[6],None,0.020373,6,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'emai****'}
2143,네이버_종목토론실,미래에셋증권,20211230-223000,왜 출금이 안되지.미래에셋증권 쓰는데[1],None,0.005959,1,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'kkwa****'}
2144,네이버_종목토론실,미래에셋증권,20211230-211100,개행주 현만이 쫏아내지 않는한,None,0.007881,0,https://finance.naver.com//item/board_read.nav...,20220817-144647,{'WRITER': 'syju****'}


In [16]:
df_ttl = pd.DataFrame()
board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=006800&page=" + str(board_nm)
    #         print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "미래에셋증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = datetime.now().strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if len(df_[df_['DATETIME']==df_already['DATETIME'].max()]) >0:
        break

    print(board_nm)
if len(df_ttl) >0:
    df_ttl = df_ttl.iloc[:df_ttl[df_ttl['DATETIME']==df_already['DATETIME'].max()].index[0]]
    if len(df_ttl) >0:
        df_ttl = df_ttl.reset_index(drop=True)

        df_ttl
        df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
        print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')  , " SUCCESS  : ",len(df_ttl))
    else:
        print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " NO DATA TO UPDATE")
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " NO DATA TO UPDATE")

    


/tmp/ipykernel_3183/2493411011.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"


20220829-231557  SUCCESS  :  16


/tmp/ipykernel_3183/2493411011.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_)


In [10]:
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220830-021700,"개인-86,085주 매도 외국인 15...[2]",NaN,0.042036,2,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'hori****'}
1,네이버_종목토론실,미래에셋증권,20220829-094800,악질적인 개 쓰레기 개 악마이자 괴물이다[1],NaN,0.011900,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': '0334****'}
2,네이버_종목토론실,미래에셋증권,20220829-095200,악질적인 개 쓰레기 개 악마이자 괴물이다,NaN,0.011367,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'sang****'}
3,네이버_종목토론실,미래에셋증권,20220829-090500,미래에셋 mts 쓰레기[3],NaN,0.006128,3,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'focu****'}
4,네이버_종목토론실,미래에셋증권,20220829-090200,80개 ....[1],NaN,0.027760,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'kss3****'}
5,네이버_종목토론실,미래에셋증권,20220828-203000,과부사정은 과부가 안다고,NaN,0.131858,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'chta****'}
6,네이버_종목토론실,미래에셋증권,20220827-122500,미래에셋 -> 타증권사 이동,NaN,0.021194,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'focu****'}
7,네이버_종목토론실,미래에셋증권,20220827-074800,개행주 급사 기원!![1],NaN,0.007575,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'skhc****'}
8,네이버_종목토론실,미래에셋증권,20220826-230900,증권주를 매수한 것 자체가 주린이임[1],NaN,0.073517,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'esse****'}
9,네이버_종목토론실,미래에셋증권,20220826-185200,대우,NaN,0.076954,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'doo8****'}


In [12]:
df_ttl = df_ttl.iloc[:df_ttl[df_ttl['DATETIME']==df_already['DATETIME'].max()].index[0]]
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,미래에셋증권,20220830-021700,"개인-86,085주 매도 외국인 15...[2]",NaN,0.042036,2,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'hori****'}
1,네이버_종목토론실,미래에셋증권,20220829-094800,악질적인 개 쓰레기 개 악마이자 괴물이다[1],NaN,0.011900,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': '0334****'}
2,네이버_종목토론실,미래에셋증권,20220829-095200,악질적인 개 쓰레기 개 악마이자 괴물이다,NaN,0.011367,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'sang****'}
3,네이버_종목토론실,미래에셋증권,20220829-090500,미래에셋 mts 쓰레기[3],NaN,0.006128,3,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'focu****'}
4,네이버_종목토론실,미래에셋증권,20220829-090200,80개 ....[1],NaN,0.027760,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'kss3****'}
5,네이버_종목토론실,미래에셋증권,20220828-203000,과부사정은 과부가 안다고,NaN,0.131858,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'chta****'}
6,네이버_종목토론실,미래에셋증권,20220827-122500,미래에셋 -> 타증권사 이동,NaN,0.021194,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'focu****'}
7,네이버_종목토론실,미래에셋증권,20220827-074800,개행주 급사 기원!![1],NaN,0.007575,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'skhc****'}
8,네이버_종목토론실,미래에셋증권,20220826-230900,증권주를 매수한 것 자체가 주린이임[1],NaN,0.073517,1,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'esse****'}
9,네이버_종목토론실,미래에셋증권,20220826-185200,대우,NaN,0.076954,0,https://finance.naver.com//item/board_read.nav...,20220829-231432,{'WRITER': 'doo8****'}


In [15]:
df_ttl.iloc[:df_ttl[df_ttl['DATETIME']==df_already['DATETIME'].max()].index[0]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [13]:
if len(df_ttl) >0:
    df_ttl = df_ttl.iloc[:df_ttl[df_ttl['DATETIME']==df_already['DATETIME'].max()].index[0]]
    if len(df_ttl) >0:
        df_ttl = df_ttl.reset_index(drop=True)

        df_ttl
        df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
        print(datetime.now().strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))
    else:
        print(datetime.now().strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")
else:
    print(datetime.now().strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")



IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
df_already[df_already['DATETIME'] == df_already['DATETIME'].max()]

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
2106,네이버_종목토론실,미래에셋증권,20220825-141100,오늘 폭등 무슨 호재 있나여?[2],None,0.281904,2,https://finance.naver.com//item/board_read.nav...,20220825-052830,{'WRITER': 'asse****'}


## 키움증권

In [15]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' and RELATED_ITEM = '{}'".format('키움증권'),conn)
df_already


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,키움증권,20220831-104100,ㅈㅎㅈㅈ 주가 ㅇㄹㄱ 는중,None,0.017268,0,https://finance.naver.com//item/board_read.nav...,20220831-143642,{'WRITER': 'sumk****'}
1,네이버_종목토론실,키움증권,20220831-050600,키움증권 해외환전 주의,None,0.026675,0,https://finance.naver.com//item/board_read.nav...,20220831-143642,{'WRITER': 'mcms****'}
2,네이버_종목토론실,키움증권,20220830-204900,오랫만에 살까 생각이 들었는데,None,0.185029,0,https://finance.naver.com//item/board_read.nav...,20220831-143642,{'WRITER': 'hidd****'}
3,네이버_종목토론실,키움증권,20220830-142000,-34%,None,0.011278,0,https://finance.naver.com//item/board_read.nav...,20220831-143642,{'WRITER': 'kjsk****'}
4,네이버_종목토론실,키움증권,20220830-112500,증권사 문제는 없는것 같다.,None,0.377894,0,https://finance.naver.com//item/board_read.nav...,20220831-143642,{'WRITER': 'sigo****'}
...,...,...,...,...,...,...,...,...,...,...
195,네이버_종목토론실,키움증권,20220603-084400,왜 키움앱 접속안됨??,None,0.065910,0,https://finance.naver.com//item/board_read.nav...,20220831-143657,{'WRITER': 'craz****'}
196,네이버_종목토론실,키움증권,20220531-130100,키웅 짱이네~[1],None,85.680850,1,https://finance.naver.com//item/board_read.nav...,20220831-143657,{'WRITER': 'og_p****'}
197,네이버_종목토론실,키움증권,20220531-100700,이거사면. 3대가망한다[2],None,0.004934,2,https://finance.naver.com//item/board_read.nav...,20220831-143657,{'WRITER': 'gdp3****'}
198,네이버_종목토론실,키움증권,20220531-094200,******자사주 매입과 cb만기****...,None,0.080146,0,https://finance.naver.com//item/board_read.nav...,20220831-143657,{'WRITER': 'ssj0****'}


In [16]:

df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=" + "039490" +"&page=" + str(board_nm)
    print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "키움증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
        break  

#     print(board_nm)






https://finance.naver.com/item/board.naver?code=039490&page=1


/tmp/ipykernel_62535/1680686252.py:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
/tmp/ipykernel_62535/1680686252.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ttl = df_ttl.append(df_)


In [17]:
df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
df_ttl  

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS


In [ ]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")



## nh투자증권

In [6]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' and RELATED_ITEM = '{}'".format('NH투자증권'),conn)
df_already


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,NH투자증권,20220831-145900,증권주에 대하여,None,0.021991,0,https://finance.naver.com//item/board_read.nav...,20220831-154054,{'WRITER': 'yji2****'}
1,네이버_종목토론실,NH투자증권,20220830-091800,n투증권을 한5만주사모아라. 그리고 배당...[10],None,0.043430,10,https://finance.naver.com//item/board_read.nav...,20220831-154054,{'WRITER': 'yoll****'}
2,네이버_종목토론실,NH투자증권,20220830-091400,농협금융의 특수성때문에 연말배당이 수월찮...[2],None,1.814806,2,https://finance.naver.com//item/board_read.nav...,20220831-154054,{'WRITER': 'yoll****'}
3,네이버_종목토론실,NH투자증권,20220829-135500,직원만족도 국내기업 3위 증권회사정리 1...[2],None,0.103602,2,https://finance.naver.com//item/board_read.nav...,20220831-154054,{'WRITER': 'chun****'}
4,네이버_종목토론실,NH투자증권,20220829-092000,파월 통수; + 환율 현재 1350;[1],None,0.125092,1,https://finance.naver.com//item/board_read.nav...,20220831-154054,{'WRITER': 'sreo****'}
...,...,...,...,...,...,...,...,...,...,...
395,네이버_종목토론실,NH투자증권,20220613-145000,질문[2],None,0.014224,2,https://finance.naver.com//item/board_read.nav...,20220831-154126,{'WRITER': 'zxcv****'}
396,네이버_종목토론실,NH투자증권,20220613-143900,버티자1[1],None,0.024904,1,https://finance.naver.com//item/board_read.nav...,20220831-154126,{'WRITER': 'qu19****'}
397,네이버_종목토론실,NH투자증권,20220613-143600,와 대박..[2],None,85.664380,2,https://finance.naver.com//item/board_read.nav...,20220831-154126,{'WRITER': 'kamj****'}
398,네이버_종목토론실,NH투자증권,20220613-142400,드디어 오늘 1만원 깨고,None,0.598647,0,https://finance.naver.com//item/board_read.nav...,20220831-154126,{'WRITER': 'ssun****'}


In [8]:

import warnings
warnings.filterwarnings("ignore")

In [9]:

df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=" + "005940" +"&page=" + str(board_nm)
    print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "NH투자증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
        break  
#     if board_nm == 20:
#         break
#     print(board_nm)



df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
df_ttl  


https://finance.naver.com/item/board.naver?code=005940&page=1


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS


In [10]:
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS


In [4]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , "{} SUCCESS  : ".format('NH투자증권'),len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), "{} NO DATA TO UPDATE".format('NH투자증권'))

20220831-154312 NH투자증권 SUCCESS  :  400


## 삼성증권

In [7]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' and RELATED_ITEM = '{}'".format('키움증권'),conn)
df_already


20220831-143717  SUCCESS  :  200


In [23]:

df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=" + "016360" +"&page=" + str(board_nm)
    print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "삼성증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
#     if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
#         break  
    if board_nm == 20:
        break
#     print(board_nm)






https://finance.naver.com/item/board.naver?code=016360&page=1
https://finance.naver.com/item/board.naver?code=016360&page=2
https://finance.naver.com/item/board.naver?code=016360&page=3
https://finance.naver.com/item/board.naver?code=016360&page=4
https://finance.naver.com/item/board.naver?code=016360&page=5
https://finance.naver.com/item/board.naver?code=016360&page=6
https://finance.naver.com/item/board.naver?code=016360&page=7
https://finance.naver.com/item/board.naver?code=016360&page=8
https://finance.naver.com/item/board.naver?code=016360&page=9
https://finance.naver.com/item/board.naver?code=016360&page=10
https://finance.naver.com/item/board.naver?code=016360&page=11
https://finance.naver.com/item/board.naver?code=016360&page=12
https://finance.naver.com/item/board.naver?code=016360&page=13
https://finance.naver.com/item/board.naver?code=016360&page=14
https://finance.naver.com/item/board.naver?code=016360&page=15
https://finance.naver.com/item/board.naver?code=016360&page=16
h

In [24]:
df_ttl

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,삼성증권,20220831-144500,3천주,NaN,0.032508,0,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'shp2****'}
1,네이버_종목토론실,삼성증권,20220831-143800,팔고 갑니다![2],NaN,0.177607,2,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'ims8****'}
2,네이버_종목토론실,삼성증권,20220831-134400,3분기 순이익[1],NaN,0.085623,1,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'sy00****'}
3,네이버_종목토론실,삼성증권,20220831-131200,오늘도 추매하기 좋은 날,NaN,99.878270,0,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'hear****'}
4,네이버_종목토론실,삼성증권,20220831-122500,신중히 봐야 할 듯[1],NaN,0.019025,1,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'moar****'}
...,...,...,...,...,...,...,...,...,...,...
15,네이버_종목토론실,삼성증권,20220712-151800,이거 너무하는거 아니냐고,NaN,0.003591,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'wkdg****'}
16,네이버_종목토론실,삼성증권,20220712-173600,[삭제된 게시물의 답글]근디 어쩐다고.주...,NaN,0.094799,삭제된 게시물의 답글근디 어쩐다고.주...,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': '3183****'}
17,네이버_종목토론실,삼성증권,20220712-150100,이제 매수하면 되나요?,NaN,0.080383,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'kang****'}
18,네이버_종목토론실,삼성증권,20220712-145300,삼성,NaN,0.036828,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'yong****'}


In [25]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), " NO DATA TO UPDATE")



20220831-145119  SUCCESS  :  400


In [26]:
import sqlite3 
conn = sqlite3.connect("/home/lsi8505/miraeasset/crawling/opinion_outside.db")
cur = conn.cursor()

import re
from bs4 import BeautifulSoup
import requests
import itertools
from selenium.webdriver.support.ui import Select
import time
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값


def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
df_already= pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE  = '네이버_종목토론실' and RELATED_ITEM = '{}'".format('삼성증권'),conn)
df_already


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,네이버_종목토론실,삼성증권,20220831-144500,3천주,None,0.032508,0,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'shp2****'}
1,네이버_종목토론실,삼성증권,20220831-143800,팔고 갑니다![2],None,0.177607,2,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'ims8****'}
2,네이버_종목토론실,삼성증권,20220831-134400,3분기 순이익[1],None,0.085623,1,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'sy00****'}
3,네이버_종목토론실,삼성증권,20220831-131200,오늘도 추매하기 좋은 날,None,99.878270,0,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'hear****'}
4,네이버_종목토론실,삼성증권,20220831-122500,신중히 봐야 할 듯[1],None,0.019025,1,https://finance.naver.com//item/board_read.nav...,20220831-145034,{'WRITER': 'moar****'}
...,...,...,...,...,...,...,...,...,...,...
395,네이버_종목토론실,삼성증권,20220712-151800,이거 너무하는거 아니냐고,None,0.003591,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'wkdg****'}
396,네이버_종목토론실,삼성증권,20220712-173600,[삭제된 게시물의 답글]근디 어쩐다고.주...,None,0.094799,삭제된 게시물의 답글근디 어쩐다고.주...,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': '3183****'}
397,네이버_종목토론실,삼성증권,20220712-150100,이제 매수하면 되나요?,None,0.080383,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'kang****'}
398,네이버_종목토론실,삼성증권,20220712-145300,삼성,None,0.036828,0,https://finance.naver.com//item/board_read.nav...,20220831-145106,{'WRITER': 'yong****'}


In [27]:

df_ttl = pd.DataFrame()

board_nm = 0

while True:
    board_nm +=1
    item_board_url = "https://finance.naver.com/item/board.naver?code=" + "016360" +"&page=" + str(board_nm)
    print(item_board_url)
    res = requests.get(item_board_url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'})#, params=params)
    soup = BeautifulSoup(res.text, "html5lib")
    #         tab_title = soup.findAll('td', attrs={'class':'title'})   
    tab_date = soup.findAll('span', attrs={'class':'tah p10 gray03'})  
    tab_date_txt = [i.text for i in tab_date]

    tab_title = soup.findAll('td', attrs={'class':'title'})  
    tab_title_txt = [i.text for i in tab_title]

    tab_writer = soup.findAll('td', attrs={'class':'p11'})  
    tab_writer_txt = [i.text for i in tab_writer]
    tab_writer_txt = tab_writer_txt[:len(tab_title_txt)]

    url_l = []
    for n in soup.findAll('td', attrs={'class':'title'})  :
        url_l.append("https://finance.naver.com/" +n.findAll('a')[0]['href'])
    url_l
    
    like_l = []
    for s in soup.findAll('tr'): 
        if len(s.findAll('strong')) > 0:
            like_l.append(s.findAll('strong')[0].text)
    like_l[:20]


    df_ = pd.DataFrame()
    df_['DATETIME'] = pd.Series(tab_date_txt).iloc[::2].reset_index(drop=True)
    df_['DATETIME'] =df_['DATETIME'].str.replace(".","").str.replace(" ","-").str.replace(":","")+"00"
    df_['VIEWS'] = pd.Series(tab_date_txt).iloc[1::2].reset_index(drop=True)
    df_['LIKES'] = like_l[:20]
    df_['TITLE'] =  pd.Series(tab_title_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)
    df_['CONTENT'] = np.nan
    df_['WRITER'] = pd.Series(tab_writer_txt).str.replace('\n','').str.replace('\t','').reset_index(drop=True)

    df_['SOURCE'] = "네이버_종목토론실"
    df_['RELATED_ITEM'] = "삼성증권"
    df_['POSITIVITY'] = np.nan
    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] = df_['TITLE'].apply(lambda x: x.split('[')[1].replace("]","") if len(x.split('[')) == 2 else 0)
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  url_l
    df_


    for i in range(len(df_)):
        df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    

    df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
    df_
    df_ttl = df_ttl.append(df_)
    if float(df_already['DATETIME'].max().replace("-","")) > float(df_ttl['DATETIME'].min().replace("-","")):
        break  
#     if board_nm == 20:
#         break
#     print(board_nm)



df_ttl = df_ttl[df_ttl['DATETIME'].str.replace("-","").astype(float) > float(df_already['DATETIME'].max().replace("-",""))]
df_ttl  


https://finance.naver.com/item/board.naver?code=016360&page=1


In [29]:
if len(df_ttl) >0:
    df_ttl = df_ttl.reset_index(drop=True)

    df_ttl
    df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , "{} SUCCESS  : ".format('삼성증권'),len(df_ttl))
else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S'), "{} NO DATA TO UPDATE".format('삼성증권'))

20220831-145246 삼성증권 NO DATA TO UPDATE
